In [1]:
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from sklearn.metrics import accuracy_score

In [2]:
# Load the datasets
train_data = pd.read_csv('tweets-ext.csv')
test_data = pd.read_csv('tweets-test.csv')
val_data = pd.read_csv('tweets-valid.csv')

In [3]:
# Preprocess the data
X_train = train_data['tweet']
y_train = train_data['label']
X_test = test_data['tweet']
y_test = test_data['label']
X_val = val_data['tweet']
y_val = val_data['label']

In [4]:
# Tokenize the text
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)

In [5]:
# Convert text sequences to numerical sequences
train_sequences = tokenizer.texts_to_sequences(X_train)
test_sequences = tokenizer.texts_to_sequences(X_test)
val_sequences = tokenizer.texts_to_sequences(X_val)

In [6]:
# Pad the sequences to a fixed length
max_seq_length = 280
train_X = pad_sequences(train_sequences, maxlen=max_seq_length)
test_X = pad_sequences(test_sequences, maxlen=max_seq_length)
val_X = pad_sequences(val_sequences, maxlen=max_seq_length)

In [7]:
# Define the ensemble models
num_models = 5
model_list = []

In [8]:
for _ in range(num_models):
    model = Sequential()
    model.add(Embedding(input_dim=len(tokenizer.word_index)+1, output_dim=100, input_length=max_seq_length))
    model.add(LSTM(units=128, return_sequences=True))
    model.add(LSTM(units=64))
    model.add(Dense(units=64, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(units=1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.fit(train_X, y_train, validation_data=(val_X, y_val), epochs=10, batch_size=32)
    model_list.append(model)

Epoch 1/10
458/458 [==============================] - 285s 588ms/step - loss: -45.6885 - accuracy: 0.4660 - val_loss: -163.3452 - val_accuracy: 0.5007
Epoch 2/10
458/458 [==============================] - 301s 656ms/step - loss: -455.4096 - accuracy: 0.5451 - val_loss: -877.8502 - val_accuracy: 0.4407
Epoch 3/10
458/458 [==============================] - 367s 801ms/step - loss: -1419.9233 - accuracy: 0.5986 - val_loss: -1931.8890 - val_accuracy: 0.4807
Epoch 4/10
458/458 [==============================] - 429s 937ms/step - loss: -2917.7478 - accuracy: 0.6519 - val_loss: -3548.2612 - val_accuracy: 0.5033
Epoch 5/10
458/458 [==============================] - 541s 1s/step - loss: -4752.8804 - accuracy: 0.6601 - val_loss: -4989.6572 - val_accuracy: 0.5033
Epoch 6/10
458/458 [==============================] - 373s 815ms/step - loss: -6928.6343 - accuracy: 0.6753 - val_loss: -6369.2227 - val_accuracy: 0.5053
Epoch 7/10
458/458 [==============================] - 376s 822ms/step - loss: -9384.

In [11]:
# Generate predictions for validation data
val_predictions = []
for model in model_list:
    predictions = model.predict(val_X)
    predicted_classes = np.argmax(predictions, axis=1)
    val_predictions.append(predicted_classes)

47/47 [==============================] - 8s 125ms/step


In [12]:
# Combine predictions using majority voting
ensemble_val_predictions = np.round(np.mean(val_predictions, axis=0))

In [13]:
# Evaluate the ensemble predictions
ensemble_val_accuracy = accuracy_score(y_val, ensemble_val_predictions)

In [14]:
print("Ensemble Validation Accuracy:", ensemble_val_accuracy)

Ensemble Validation Accuracy: 0.3333333333333333


In [16]:
# Generate predictions for test data
test_predictions = []
for model in model_list:
    predictions = model.predict(test_X)
    predicted_classes = np.argmax(predictions, axis=1)
    test_predictions.append(predicted_classes)

71/71 [==============================] - 9s 125ms/step


In [17]:
# Combine predictions using majority voting
ensemble_test_predictions = np.round(np.mean(test_predictions, axis=0))

In [18]:
# Evaluate the ensemble predictions
ensemble_test_accuracy = accuracy_score(y_test, ensemble_test_predictions)

In [19]:
print("Ensemble Test Accuracy:", ensemble_test_accuracy)

Ensemble Test Accuracy: 0.3333333333333333
